In [1]:
ASEMAT = ["Rankki",
          "Orrengrund"
]

def parse_direction(deg):
    suunnat = ["pohjoinen",
               "pohjoiskoillinen",
               "koillinen",
               "itäkoillinen",
               "itä",
               "itäkaakko",
               "kaakko",
               "eteläkaakko",
               "etelä",
               "etelälounas",
               "lounas",
               "länsilounas",
               "länsi",
               "länsiluode",
               "luode",
               "pohjoisluode",
               "pohjoinen"]
    return suunnat[round(deg/22.5)]

weather_symbol_map = {1: "selkeää",
        2: "puolipilvistä",
        21: "heikkoja sadekuuroja",
        22: "sadekuuroja",
        23: "voimakkaita sadekuuroja",
        3: "pilvistä",
        31: "heikkoa vesisadetta",
        32: "vesisadetta",
        33: "voimakasta vesisadetta",
        41: "heikkoja lumikuuroja",
        42: "lumikuuroja",
        43: "voimakkaita lumikuuroja",
        51: "heikkoa lumisadetta",
        52: "lumisadetta",
        53: "voimakasta lumisadetta",
        61: "ukkoskuuroja",
        62: "voimakkaita ukkoskuuroja",
        63: "ukkosta",
        64: "voimakasta ukkosta",
        71: "heikkoja räntäkuuroja",
        72: "räntäkuuroja",
        73: "voimakkaita räntäkuuroja",
        81: "heikkoa räntäsadetta",
        82: "räntäsadetta",
        83: "voimakasta räntäsadetta",
        91: "utua",
        92: "sumua"}


def numero_tekstiksi(n):
    """Muuttaa kokonaisluvun välillä -9999 - 9999 tekstiksi"""
    numero_map = {1: "yksi",
                2: "kaksi",
                3: "kolme",
                4: "neljä",
                5: "viisi",
                6: "kuusi",
                7: "seitsemän",
                8: "kahdeksan",
                9: "yhdeksän"}
    if abs(n) >= 10000:
        raise Exception("Vain luvut -9999 - 9999")
    if n == 0:
        return "nolla"
    if n < 0:
        ret = "miinus "
        n = abs(n)
    else:
        ret = ""
    if n < 10:
        return ret + numero_map[n]
    if n == 10:
        return ret + "kymmenen"
    if (n > 10) and (n < 20):
        r = n%10
        return ret + numero_map[r] + "toista"
    
    tuhannet = n//1000
    sadat = (n- 1000*tuhannet)//100
    kymmenet = (n - 1000*tuhannet - 100*sadat)//10
    yhdet = n%10

    tuhat_s = sadat_s = kymmen_s = yhdet_s = ""
    
    if yhdet != 0:
        yhdet_s = numero_map[yhdet]

    if kymmenet != 0:
        if kymmenet == 1:
            kymmen_s = numero_tekstiksi(kymmenet*10 + yhdet)
            yhdet_s = ""
        else:
            kymmen_s = numero_map[kymmenet] + "kymmentä"

    if sadat != 0:
        if sadat == 1:
            sadat_s = "sata"
        else:
            sadat_s = numero_map[sadat] + "sataa"

    if tuhannet != 0:
        if tuhannet == 1:
            tuhat_s = "tuhat"
        else:
            tuhat_s = numero_map[tuhannet] + "tuhatta"
    
    return ret + tuhat_s + sadat_s + kymmen_s + yhdet_s

In [2]:
from fmiopendata.wfs import download_stored_query
import datetime as dt


def get_asema_obs(asema):
      end_time = dt.datetime.utcnow()
      start_time = end_time - dt.timedelta(days=1)

      start_time = start_time.isoformat(timespec="seconds") + "Z"
      end_time = end_time.isoformat(timespec="seconds") + "Z"

      return download_stored_query("fmi::observations::weather::hourly::multipointcoverage",
                        args=["place=" + asema,
                              "starttime=" + start_time,
                              "endtime=" + end_time])

obs = get_asema_obs("Tankar")

In [3]:
latest_step = max(obs.data.keys())
place = list(obs.data[latest_step].keys())
if len(place) == 1:
    place = place[0]
else:
    pass
data = obs.data[latest_step][place]
print(latest_step)
print(place)

for key in data.keys():
    print(key)
    print(data[key])

2023-10-07 13:00:00
Kokkola Tankar
Air temperature
{'value': 6.0, 'units': 'degC'}
Highest temperature
{'value': 6.5, 'units': 'degC'}
Lowest temperature
{'value': 5.4, 'units': 'degC'}
Relative humidity
{'value': 77.0, 'units': '%'}
Wind speed
{'value': 18.0, 'units': 'm/s'}
Maximum wind speed
{'value': 19.1, 'units': 'm/s'}
Minimum wind speed
{'value': 17.0, 'units': 'm/s'}
Wind direction
{'value': 350.0, 'units': 'deg'}
Precipitation amount
{'value': nan, 'units': 'mm'}
Maximum precipitation intensity
{'value': nan, 'units': 'mm/h'}
Air pressure
{'value': 993.8, 'units': 'hPa'}
Present weather (auto)
{'value': 81.0, 'units': ''}


In [4]:
import numpy as np
def get_asema(asema):
    obs = get_asema_obs(asema)

    latest_step = max(obs.data.keys())
    place = list(obs.data[latest_step].keys())[0]
    data = obs.data[latest_step][place]

    temp = numero_tekstiksi(round(data["Air temperature"]["value"]))
    wind_dir = parse_direction(data["Wind direction"]["value"])
    wind_speed = numero_tekstiksi(round(data["Wind speed"]["value"]))
    # try:
    #     weather = weather_symbol_map[data["Present weather (auto)"]["value"]]
    # except Exception:
    #     weather = data["Present weather (auto)"]["value"]
    #     if np.isnan(weather):
    #         weather = ""
    return f"{asema} {temp}, {wind_dir} {wind_speed}"

In [5]:
"Harmaja",
"Emäsalo",
"Itätoukki",
"Bågaskär",
"Russarö",
"Vänö",
"Utö",
"Bogskär",
"Rajakari",

('Rajakari',)

In [6]:
get_asema("Haapasaari")

'Haapasaari kuusi, pohjoisluode kaksitoista'

In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from pprint import pprint
from bs4 import BeautifulSoup


In [171]:
driver = webdriver.Chrome()

In [9]:
driver.get("https://www.ilmatieteenlaitos.fi/rannikkosaa?station=101042")

In [142]:
# Ladataan rannikkoasemien koodit
# html = driver.page_source
# soup = BeautifulSoup(html)
# long2value = {}
# for asema_section in list(soup.select("select[aria-label^='Valitse havaintoasema, jonka havainnot näytetään alapuolella']")[0]):
#     asema = asema_section.get_text().lower()
#     value = asema_section["value"]
#     long2value[asema] = value
# long2value

# with open("rannikkoasemat.txt", "w") as f:
#     for k,v in long2value.items():
#         f.write(f"{k}, {v}\n")

In [143]:
with open("rannikkoasemat.txt", "r") as f:
    lines = f.readlines()
short2value = {x.split(",")[0].strip(): x.split(",")[1].strip() for x in lines}
short2value

{'haapasaari': '101042',
 'kotka rankki': '101030',
 'orrengrund': '101039',
 'emäsalo': '101023',
 'kalbådagrund': '101022',
 'itätoukki': '105392',
 'harmaja': '100996',
 'mäkiluoto': '100997',
 'bågaskär': '100969',
 'jussarö': '100965',
 'hanko tulliniemi': '100946',
 'russarö': '100932',
 'vänö': '100945',
 'utö': '100908',
 'bogskär': '100921',
 'rajakari': '100947',
 'fagerholm': '100924',
 'kumlinge': '100928',
 'nyhamn': '100909',
 'märket': '100919',
 'isokari': '101059',
 'kylmäpihlaja': '101061',
 'tahkoluoto': '101267',
 'sälgrund': '101256',
 'bredskäret': '101479',
 'strömmingsbådan': '101481',
 'valassaaret': '101464',
 'kallan': '101660',
 'tankar': '101661',
 'ulkokalla': '101673',
 'nahkiainen': '101775',
 'raahe': '101785',
 'oulu vihreäsaari': '101794',
 'marjaniemi': '101784',
 'kemi ykkönen': '101783',
 'ajos': '101846'}

In [151]:
def load_page(driver, station_id):
    driver.get(f"https://www.ilmatieteenlaitos.fi/rannikkosaa?station={station_id}")
    wait = WebDriverWait(driver, timeout=5)
    data_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div[data-v-e11b1aae].row")))

def load_raw_data(driver):
    """Scrapeaa rannikkoaseman viimeisimmät havainnot ja palauttaa ne sellaisessa muodossa kuin ne ovat sivulla"""

    # Lataa sivun html
    html = driver.page_source
    soup = BeautifulSoup(html)

    # Hae havainnot
    havainnot_section = soup.select("section[aria-label^='Viimeisimmät säähavainnot']")
    havainnot = list(havainnot_section[0])[0]

    data_section = list(havainnot.children)[2]
    aikaleima_raw = list(havainnot.children)[0].get_text()
    data_raw = [x.get_text() for x in data_section.children]
    return aikaleima_raw, data_raw


In [145]:
def parse_aikaleima(aikaleima_raw):
    """Muuttaa verkkosivulta poimitun aikaleiman luettavaksi tekstiksi
    esim 'Viimeisin säähavainto: La 7.10. 14:40  ' -> 'neljätoista neljäkymmentä'
    """
    aika = aikaleima_raw.split(" ")[4]
    tunti, minuutti = aika.split(":")
    return numero_tekstiksi(int(tunti)) + " " + numero_tekstiksi(int(minuutti))


In [156]:
load_page(driver, short2value["sälgrund"])
aika, data = load_raw_data(driver)

In [163]:
data_dict = create_data_dict(data)
parse_data_dict(data_dict)

'kuusi pohjoisluode neljätoista sadetta melkein pilvistä hyvä näkyvyys'

In [158]:
def create_data_dict(data_raw):
    data_vars = ['Lämpötila',
            'Kastepiste',
            'Kosteus',
            'Tuuli',
            'Tuulen suunta',
            'Tuulen puuska',
            'Näkyvyys',
            'Pilvisyys',
            'Paine',
            'Sade']

    data_dict = {}
    for v in data_vars:
        data_dict[v] = None
        for data_row in data_raw:
            if data_row.startswith(v):
                data_dict[v] = data_row.replace(v, "").strip()
    return data_dict

In [170]:
import re
def parse_data_dict(data_dict):
    lampotila = round(float(data_dict["Lämpötila"].split(" ")[0].replace(",", ".")))
    lampotila_str = numero_tekstiksi(lampotila)

    tuuli = int(data_dict["Tuuli"].split(" ")[0])
    tuuli_str = numero_tekstiksi(tuuli)

    tuulen_suunta = int(re.search(r"\((\d+)\°\)", data_dict["Tuulen suunta"]).group(1))
    tuulen_suunta_str = parse_direction(tuulen_suunta)

    if data_dict["Sade"] is None:
        sade_str = ""
    else:
        sade_str = "sadetta"

    if data_dict["Pilvisyys"] is None:
        pilvisyys_str = ""
    else:
        pilvisyys_str = " ".join(data_dict["Pilvisyys"].split(" ")[:-1]).lower()

    if data_dict["Näkyvyys"] is None:
        nakyvyys_str = ""
    else:
        nakyvyys = data_dict["Näkyvyys"].split(" ")[0]
        if nakyvyys == "yli":
            nakyvyys_str = "hyvä näkyvyys"
        else:
            nakyvyys = round(float(nakyvyys.replace(",", ".")))
            nakyvyys_str = numero_tekstiksi(nakyvyys)
        

    asema_str = f"{lampotila_str} {tuulen_suunta_str} {tuuli_str} {sade_str} {pilvisyys_str} {nakyvyys_str}"
    return asema_str

load_page(driver, short2value["bågaskär"])

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=117.0.5938.150)
Stacktrace:
	GetHandleVerifier [0x00007FF6C5D37D12+55474]
	(No symbol) [0x00007FF6C5CA77C2]
	(No symbol) [0x00007FF6C5B5E0EB]
	(No symbol) [0x00007FF6C5B3E528]
	(No symbol) [0x00007FF6C5BC3B77]
	(No symbol) [0x00007FF6C5BD75BF]
	(No symbol) [0x00007FF6C5BBEF33]
	(No symbol) [0x00007FF6C5B93D41]
	(No symbol) [0x00007FF6C5B94F84]
	GetHandleVerifier [0x00007FF6C609B762+3609346]
	GetHandleVerifier [0x00007FF6C60F1A80+3962400]
	GetHandleVerifier [0x00007FF6C60E9F0F+3930799]
	GetHandleVerifier [0x00007FF6C5DD3CA6+694342]
	(No symbol) [0x00007FF6C5CB2218]
	(No symbol) [0x00007FF6C5CAE484]
	(No symbol) [0x00007FF6C5CAE5B2]
	(No symbol) [0x00007FF6C5C9EE13]
	BaseThreadInitThunk [0x00007FFE89067344+20]
	RtlUserThreadStart [0x00007FFE892C26B1+33]


In [152]:
from tqdm.notebook import tqdm
from time import sleep
def rannikkoasema_pipeline(asema):
    try:
        load_page(driver, short2value[asema])
        aikaleima_raw, data_raw = load_raw_data(driver)
        if len(data_raw) == 0:
            sleep(2)
            aikaleima_raw, data_raw = load_raw_data(driver)
        data_dict = create_data_dict(data_raw)
        aikaleima_str = parse_aikaleima(aikaleima_raw)
        asema_str = parse_data_dict(data_dict)
    except Exception:
        return "", "ei havaintoja"
    return aikaleima_str, asema_str

for i, asema in enumerate(tqdm(short2value.keys())):
    aikaleima_str, asema_str = rannikkoasema_pipeline(asema)
    if i == 0:
        print(f"Sää rannikkoasemilla tänään kello {aikaleima_str}")
    print(f"{asema}: {asema_str}.")
    sleep(1)

  0%|          | 0/36 [00:00<?, ?it/s]

Sää rannikkoasemilla tänään kello seitsemäntoista viisikymmentä
haapasaari: viisi luode yhdeksäntoista pilvistä kymmenen
kotka rankki: ei havaintoja
orrengrund: kuusi luode seitsemäntoista melkein pilvistä hyvä näkyvyys
emäsalo: kuusi luode kuusitoista  
kalbådagrund: kahdeksan luode kaksikymmentäkaksi  
itätoukki: kahdeksan luode kuusitoista  
harmaja: seitsemän luode seitsemäntoista melkein pilvistä 
mäkiluoto: seitsemän luode kaksikymmentäkaksi  
bågaskär: seitsemän pohjoisluode yksitoista melkein pilvistä hyvä näkyvyys
jussarö: kahdeksan luode yksitoista melkein selkeää hyvä näkyvyys
hanko tulliniemi: seitsemän luode kaksikymmentä  
russarö: kahdeksan luode kaksikymmentä  
vänö: kahdeksan pohjoisluode seitsemäntoista  
utö: kahdeksan luode seitsemäntoista puolipilvistä kolmekymmentä
bogskär: kahdeksan pohjoisluode kaksikymmentäneljä  
rajakari: kuusi luode kahdeksantoista  
fagerholm: kahdeksan luode seitsemäntoista  
kumlinge: kuusi pohjoisluode seitsemäntoista melkein pilvistä ka

In [153]:
kaikki_asemat ="""
Sää rannikkoasemilla tänään kello seitsemäntoista viisikymmentä
haapasaari: viisi luode yhdeksäntoista pilvistä kymmenen
kotka rankki: ei havaintoja
orrengrund: kuusi luode seitsemäntoista melkein pilvistä hyvä näkyvyys
emäsalo: kuusi luode kuusitoista  
kalbådagrund: kahdeksan luode kaksikymmentäkaksi  
itätoukki: kahdeksan luode kuusitoista  
harmaja: seitsemän luode seitsemäntoista melkein pilvistä 
mäkiluoto: seitsemän luode kaksikymmentäkaksi  
bågaskär: seitsemän pohjoisluode yksitoista melkein pilvistä hyvä näkyvyys
jussarö: kahdeksan luode yksitoista melkein selkeää hyvä näkyvyys
hanko tulliniemi: seitsemän luode kaksikymmentä  
russarö: kahdeksan luode kaksikymmentä  
vänö: kahdeksan pohjoisluode seitsemäntoista  
utö: kahdeksan luode seitsemäntoista puolipilvistä kolmekymmentä
bogskär: kahdeksan pohjoisluode kaksikymmentäneljä  
rajakari: kuusi luode kahdeksantoista  
fagerholm: kahdeksan luode seitsemäntoista  
kumlinge: kuusi pohjoisluode seitsemäntoista melkein pilvistä kahdeksantoista
nyhamn: seitsemän pohjoisluode yhdeksäntoista puolipilvistä kuusitoista
märket: kahdeksan luode kaksikymmentä  
isokari: seitsemän luode kaksikymmentäyksi puolipilvistä kuusitoista
kylmäpihlaja: kahdeksan pohjoisluode kaksikymmentäneljä puolipilvistä yksitoista
tahkoluoto: kahdeksan pohjoisluode kaksikymmentä melkein pilvistä kahdeksan
sälgrund: kuusi pohjoisluode neljätoista melkein pilvistä hyvä näkyvyys
bredskäret: kuusi pohjoisluode kaksitoista  
strömmingsbådan: kuusi pohjoisluode kuusitoista  
valassaaret: viisi pohjoisluode viisitoista  
kallan: kuusi pohjoisluode yhdeksäntoista  
tankar: kuusi pohjoinen kahdeksantoista melkein pilvistä kolmetoista
ulkokalla: kuusi pohjoisluode kuusitoista  
nahkiainen: neljä pohjoinen seitsemäntoista  
raahe: neljä pohjoinen seitsemäntoista  kahdeksan
oulu vihreäsaari: kaksi pohjoinen kymmenen  kuusi
marjaniemi: neljä pohjoiskoillinen kahdeksantoista pilvistä kaksikymmentä
kemi ykkönen: neljä pohjoinen neljätoista  
ajos: kolme pohjoinen yhdeksän melkein pilvistä hyvä näkyvyys
"""

In [154]:
from TTS.api import TTS

tts = TTS("tts_models/fin/fairseq/vits")
tts.tts_to_file(text= kaikki_asemat,
                file_path="asemat.wav")

 > tts_models/fin/fairseq/vits is already downloaded.
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024


C:\Users\mikko\AppData\Roaming\Python\Python39\site-packages\torch\nn\utils\weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


 > Text splitted to sentences.
['Sää rannikkoasemilla tänään kello seitsemäntoista viisikymmentä', 'haapasaari: viisi luode yhdeksäntoista pilvistä kymmenen', 'kotka rankki: ei havaintoja', 'orrengrund: kuusi luode seitsemäntoista melkein pilvistä hyvä näkyvyys', 'emäsalo: kuusi luode kuusitoista  kalbådagrund: kahdeksan luode kaksikymmentäkaksi  itätoukki: kahdeksan luode kuusitoista  harmaja: seitsemän luode seitsemäntoista melkein pilvistä mäkiluoto: seitsemän luode kaksikymmentäkaksi  bågaskär: seitsemän pohjoisluode yksitoista melkein pilvistä hyvä näkyvyys', 'jussarö: kahdeksan luode yksitoista melkein selkeää hyvä näkyvyys', 'hanko tulliniemi: seitsemän luode kaksikymmentä  russarö: kahdeksan luode kaksikymmentä  vänö: kahdeksan pohjoisluode seitsemäntoista  utö: kahdeksan luode seitsemäntoista puolipilvistä kolmekymmentä', 'bogskär: kahdeksan pohjoisluode kaksikymmentäneljä  rajakari: kuusi luode kahdeksantoista  fagerholm: kahdeksan luode seitsemäntoista  kumlinge: kuusi pohjo

'asemat.wav'

In [165]:
def saatiedotus_cleaner(s):
    saatiedotus_header = s.split("\n")[0]
    aika = numero_tekstiksi(int(saatiedotus_header.split(" ")[-1]))
    s = s.replace(saatiedotus_header, f"Säätiedotus merenkulkijoille tänään kello {aika}")


    # Korjaa pilkulliset numerot. 2,5 -> "kaksi pilkku viisi"
    pilkku_numerot = re.findall(r"\b\d+,\d+\b", s)
    for p in pilkku_numerot:
        s = s.replace(p, p.replace(",", " pilkku "))

    # Korjaa ranget. 8-12 -> "8 viiva 12"
    viiva_numerot = re.findall(r"\b\d+-\d+\b", s)
    for v in viiva_numerot:
        s = s.replace(v, v.replace("-", " viiva "))
    
    s = s.replace(" klo ", " kello ")
    s = s.replace("m/s", "metriä sekunnissa")

    # korjaa numerot
    def replace_number(match):
        number = int(match.group(0))
        return numero_tekstiksi(number)

    s = re.sub(r"\d+", lambda x: replace_number(x), s)
    return s

def get_saatiedotus(driver):
    saatiedotus_url = "https://www.ilmatieteenlaitos.fi/saatiedotus-merenkulkijoille"
    driver.get(saatiedotus_url)

    soup = BeautifulSoup(driver.page_source)
    saatiedotus_string = soup.select("div[id^=proxyId_6mddLTaDJSXFZjHemVsl61]")[0].text
    
    return saatiedotus_cleaner(saatiedotus_string)

In [167]:
saatiedotus_str = get_saatiedotus(driver)
print(saatiedotus_str)

Säätiedotus merenkulkijoille tänään kello viisitoista
Myrskyvaroitus:
Suomenlahti, Pohjois-Itämeri, Ahvenanmeri, Saaristomeri ja Selkämeren eteläosa
 Pohjoisen ja luoteen välistä myrskyä kaksikymmentäkolme metriä sekunnissa.

Kovan tuulen varoitus:
Selkämeren pohjoisosa
 Pohjoisen ja luoteen välistä tuulta yhdeksäntoista metriä sekunnissa.

Merenkurkku ja Perämeren eteläosa
 Pohjoistuulta yhdeksäntoista metriä sekunnissa.

Perämeren pohjoisosa
 Pohjoistuulta seitsemäntoista metriä sekunnissa.

Huomautus veneilijöille:
Saimaa
 Pohjoisenpuoleista tuulta kaksitoista metriä sekunnissa.

Aallokkovaroitus


Suomenlahdella, Pohjois-Itämerellä, Ahvenanmerellä ja Selkämerellä esiintyy kovaa aallokkoa, jossa merkitsevä aallonkorkeus ylittää neljä metriä.

Merenkurkussa ja Perämerellä esiintyy kohtalaista aallokkoa, jossa merkitsevä aallonkorkeus ylittää kaksi pilkku viisi metriä.

Maan kaakkoisosan yllä oleva myrskymatalapaine liikkuu itään.

Odotettavissa huomiseen iltapäivään asti:
Suomenlahti

In [168]:
with open("saatiedotus_merenkulkijoille.txt", "w") as f:
    f.writelines(saatiedotus_str)

In [91]:
tts.tts_to_file(text=saatiedotus_cleaned,
                file_path="saatiedotus.wav")

 > Text splitted to sentences.
['Säätiedotus merenkulkijoille tänään kello viisitoista', 'Myrskyvaroitus:', 'Suomenlahti, Pohjois-Itämeri, Ahvenanmeri, Saaristomeri ja Selkämeren eteläosa', 'Pohjoisen ja luoteen välistä myrskyä kaksikymmentäkolme metriä sekunnissa.', 'Kovan tuulen varoitus:', 'Selkämeren pohjoisosa', 'Pohjoisen ja luoteen välistä tuulta yhdeksäntoista metriä sekunnissa.', 'Merenkurkku ja Perämeren eteläosa', 'Pohjoistuulta yhdeksäntoista metriä sekunnissa.', 'Perämeren pohjoisosa', 'Pohjoistuulta seitsemäntoista metriä sekunnissa.', 'Huomautus veneilijöille:', 'Saimaa', 'Pohjoisenpuoleista tuulta kaksitoista metriä sekunnissa.', 'Aallokkovaroitus', 'Suomenlahdella, Pohjois-Itämerellä, Ahvenanmerellä ja Selkämerellä esiintyy kovaa aallokkoa, jossa merkitsevä aallonkorkeus ylittää neljä metriä.', 'Merenkurkussa ja Perämerellä esiintyy kohtalaista aallokkoa, jossa merkitsevä aallonkorkeus ylittää kaksi pilkku viisi metriä.', 'Maan kaakkoisosan yllä oleva myrskymatalapaine

'saatiedotus.wav'